# Create balanced datasets
The goal of this Jupyter Notebook is to create two balanced datasets that can be utilized for the machine learning models.

In the first balanced dataset, the tweets will only be backtranslated. 

For the second balanced dataset, we will also randomly replaces words in tweet with synonyms.

## Import data and create variables

In [1]:
import pandas as pd
data = pd.read_pickle("~/Documents/Github Repository/early-warning-twitter/Processed datasets/Tweets/01-06-2020-unique-tweets-amsterdam-demonstration.pkl")

In [2]:
# Seperate non incident related tweets and incident related tweets
not_incident_related_data = data[data["has_Incident_Related"]==0]
incident_related_data = data[data["has_Incident_Related"]==1]

# Reset the index
not_incident_related_data.reset_index(inplace=True, drop=True)
incident_related_data.reset_index(inplace=True, drop=True)

## Backtranslation on dataset
For every Incident Related tweet, the partly processed tweet will be translated to English and French and translated back to Dutch. If the backtranslated tweet is different than the original tweet, it will be added to the dataset.

Back translation will be performed by using the Google Translate API.

In [ ]:
import os
from google.cloud import translate_v2 as translate
import time

# Get Google API Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"path-to-json-file"

# Initiate translation Client
client = translate.Client()

incident_related_back_translated = pd.DataFrame(columns=["preprocessed_text_partly", "hashtags", "has_media", "has_Incident_Related"])

for i in range(len(incident_related_data)):
    
    # Get partly preprocessed text
    text = incident_related_data.loc[i, 'preprocessed_text_partly']
    
    # Translate it to English using Google Translate API
    middle = client.translate(text, target_language='en')["translatedText"]
    
    # Translate it back to Dutch using Google Translate API
    output = client.translate(middle, target_language='nl')["translatedText"]
    
    # Check if the text is not equal to the output
    if(text != output):
        incident_related_back_translated = incident_related_back_translated.append({"preprocessed_text_partly":output, "hashtags": incident_related_data.loc[i, 'hashtags'], "has_media":incident_related_data.loc[i, 'has_media'], "has_Incident_Related":incident_related_data.loc[i, 'has_Incident_Related']}, ignore_index=True)
    
    # Translate it to French using Google Translate API
    middle2 = client.translate(text, target_language='fr')["translatedText"]
    
    # Translate it back to Dutch using Google Translate API
    output2 = client.translate(middle2, target_language='nl')["translatedText"]
    
    # Check if the text is not equal to the output
    if(text != output2) and (output != output2):
        incident_related_back_translated = incident_related_back_translated.append({"preprocessed_text_partly":output, "hashtags": incident_related_data.loc[i, 'hashtags'], "has_media":incident_related_data.loc[i, 'has_media'], "has_Incident_Related":incident_related_data.loc[i, 'has_Incident_Related']}, ignore_index=True)

In [3]:
import os
from google.cloud import translate_v2 as translate
import time

# Get Google API Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/Users/jorenwouters/Documents/TranslationAPI/GoogleCloudKey_MyServiceAcct.json"

# Initiate translation Client
client = translate.Client()

incident_related_back_translated = pd.DataFrame(columns=["preprocessed_text_partly", "hashtags", "has_media", "has_Incident_Related"])

for i in range(len(incident_related_data)):
    
    # Get partly preprocessed text
    text = incident_related_data.loc[i, 'preprocessed_text_partly']
    
    # Translate it to English using Google Translate API
    middle = client.translate(text, target_language='en')["translatedText"]
    
    # Translate it back to Dutch using Google Translate API
    output = client.translate(middle, target_language='nl')["translatedText"]
    
    # Check if the text is not equal to the output
    if(text != output):
        incident_related_back_translated = incident_related_back_translated.append({"preprocessed_text_partly":output, "hashtags": incident_related_data.loc[i, 'hashtags'], "has_media":incident_related_data.loc[i, 'has_media'], "has_Incident_Related":incident_related_data.loc[i, 'has_Incident_Related']}, ignore_index=True)
    
    # Translate it to French using Google Translate API
    middle2 = client.translate(text, target_language='fr')["translatedText"]
    
    # Translate it back to Dutch using Google Translate API
    output2 = client.translate(middle2, target_language='nl')["translatedText"]
    
    # Check if the text is not equal to the output
    if(text != output2) and (output != output2):
        incident_related_back_translated = incident_related_back_translated.append({"preprocessed_text_partly":output, "hashtags": incident_related_data.loc[i, 'hashtags'], "has_media":incident_related_data.loc[i, 'has_media'], "has_Incident_Related":incident_related_data.loc[i, 'has_Incident_Related']}, ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Replace random words in tweet with synonyms
In each tweet, 3 random words will be replaced with a synonym.
To replace words with synonyms, a request to synoniemen.net is utilized.

In [ ]:
import random
import requests
from bs4 import BeautifulSoup

synonyms = pd.DataFrame(columns=["word", "synonyms"])
not_synonyms = ['meter']

# This function randomly replaces words in a string with synonyms
# First parameter is the string itself
# Second parameter is the number of words that need to be replaced
# Third parameter is a list that contains words we will not replace
def random_replace_words_synonyms(text, n_words, exclude_words):
    
    global synonyms
    global not_synonyms
    
    # Convert the text to a list
    list_text = list(text.split(" "))
    
    # Determine the length of the list
    length_list = len(list_text)
    
    if length_list < n_words:
        while length_list < n_words:
            print("This string is larger than the number of words that needs to be replaced!", text)
            n_words = n_words - 1
    
    list_numbers = []
    for i in range(n_words):
        
        # Try to replace a word with a synonym, until you have a match
        while True:
            # Choose a random int (based on the maximum number in the list)
            random_int = random.randint(0, length_list-1)

            # Get the word that corresponds to the random int
            word_chosen = list_text[random_int]
            
            break_twice = False

            # Check if the chosen word is not already used
            # And check if the chosen word is not one of the excluded words
            if (random_int in list_numbers) or (word_chosen in exclude_words) or (word_chosen in not_synonyms):
                break;
                
            # Check if we already know the synonyms for the chosen word
            elif (synonyms['word'] == word_chosen).any() == True:
                # Get index of the row in the synonyms DataFrame
                index_word = synonyms[synonyms["word"]==word_chosen].index.values.astype(int)[0]
                
                # Get the synonyms for the word
                list_synonyms = synonyms.loc[index_word, 'synonyms']
                
                # Choose a random synonym from the list of synonyms
                synonym = list_synonyms[random.randint(0, len(list_synonyms)-1)]
                
                list_text[random_int] = synonym
            
            else:
                list_numbers.append(random_int)
                
                # Determine synonym for the chosen word
                url = f"https://synoniemen.net/index.php?zoekterm={word_chosen}"
    
                # Make a request to the URL
                reqs = requests.get(url)

                # Get the HTML of the URL
                soup = BeautifulSoup(reqs.text, 'html.parser')

                # Get the synonyms list element on the page
                synonym_list_element = soup.find('dl', {"class":"alstrefwoordtabel"})
                
                # If there are synonyms available
                if synonym_list_element != None:
                    synonym_dd = synonym_list_element.find('dd')
                    synonym_a = synonym_dd.find_all('a')
                    synonyms_list = []
                    
                    # Store every synonym in the synonyms_list
                    for element in synonym_a:
                        synonyms_list.append(element.get_text())
                    
                    # Append the list of synonyms to the synonyms DataFrame
                    synonyms = synonyms.append({"word":word_chosen, "synonyms":synonyms_list}, ignore_index=True)
                    
                    # Choose a random synonym from the list of synonyms
                    synonym = synonyms_list[random.randint(0, len(synonyms_list)-1)]
                    
                    list_text[random_int] = synonym
                
                # If there are no synonyms available
                else:
                    not_synonyms.append(word_chosen)
                    
                    # Necessary to break the while statement
                    break_twice = True
                    continue
            
            # Break the while statement
            if(break_twice==True):
                break;
    
    return list_text

incident_related_synonym_replaced = pd.DataFrame(columns=["preprocessed_text_partly", "hashtags", "has_media", "has_Incident_Related"])

for i in range(len(incident_related_data)):
    
    # Get preprocessed text
    text = incident_related_data.loc[i, 'preprocessed_text']
    
    # Replace synonyms
    output = random_replace_words_synonyms(text, 3, exclude_words=['halsema', 'at5'])
    
    # Check if the text is not equal to the output
    if(text != output):
        incident_related_synonym_replaced = incident_related_synonym_replaced.append({"preprocessed_text_partly":output, "hashtags": incident_related_data.loc[i, 'hashtags'], "has_media":incident_related_data.loc[i, 'has_media'], "has_Incident_Related":incident_related_data.loc[i, 'has_Incident_Related']}, ignore_index=True)

In [ ]:
# Check the synonyms that were used
synonyms

## Clean the two created datasets

In [6]:
# Import necessary packages
import re
import demoji
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup

# Download demoji codes
demoji.download_codes()

# Clean the text of the tweet
def clean_text(row, variable, hashtag_text='keep', representation = 'string'):
    
    # Parameters
    # hashtag_text, default = 'keep'
        # 'keep' - keeps the hashtag text and only removes the '#' in the text
        # 'lose' - both removes the hashtag text and the '#' in the text
    # representation, default = 'string'
        # 'list' - returns a list of words
        # 'string' - returns a sentence in string format
    
    tweet = row[variable]
    
    # Make the tweet lowercase
    tweet = tweet.lower()
    
    # Remove words with less than two characters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    
    # Remove URLs
    tweet = remove_url(tweet)
    
    # Remove punctuations unless they are part of a digit (such as "5.000")
    tweet = re.sub(r'(?:(?<!\d)[.,;:…‘]|[.,;:…‘](?!\d))', '', tweet)
    
    # Remove emojis
    tweet = demoji.replace(tweet, "")
    
    if hashtag_text == 'keep':
        tweet = tweet.replace("#", "")
        # Remove mentions (also the text after the @)
        tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)", "", tweet).split())
    else:
        # Remove hashtags and mentions (also the text after the # and @)
        tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", "", tweet).split())
    
    # Remove non-alphanumeric charachters, line breaks and tabs
    tweet = ' '.join(re.sub("([:/\!@#$%^&*()_+{}[\];\"”\'|?<>~`\-\n\t’])", "", tweet).split())
    
    # Tokenize the tweet
    tweet = word_tokenize(tweet)
    
    # Use Dutch stop words
    stop_words = stopwords.words('dutch') + ["rt", "nan", "NaN"] 
    
    # Remove stopwords
    tweet = [w for w in tweet if not w in stop_words]
    
    if representation == 'list':
        return tweet
    else:
        return listToString(tweet)
    
# Function to convert a list to a string
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 
    
def remove_url(tweet_text):
    if has_url_regex(tweet_text): 
        url_regex_list = regex_url_extractor(tweet_text)
        for url in url_regex_list:
            tweet_text = tweet_text.replace(url, "")
    return tweet_text

def has_url_regex(tweet_text):
    return regex_url_extractor(tweet_text)

def regex_url_extractor(tweet_text):
    return re.findall('https?:\/\/(?:[-\w\/.]|(?:%[\da-fA-F]{2}))+', tweet_text)

# Clean the tweet texts and put it in four variables per dataset
incident_related_back_translated["preprocessed_text"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "keep", "string"]), axis=1)
incident_related_back_translated["preprocessed_text_no_hashtag"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "lose", "string"]), axis=1)
incident_related_back_translated["preprocessed_text_tokenized"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "keep", "list"]), axis=1)
incident_related_back_translated["preprocessed_text_tokenized_no_hashtag"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "lose", "list"]), axis=1)

incident_related_synonym_replaced["preprocessed_text"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "keep", "string"]), axis=1)
incident_related_synonym_replaced["preprocessed_text_no_hashtag"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "lose", "string"]), axis=1)
incident_related_synonym_replaced["preprocessed_text_tokenized"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "keep", "list"]), axis=1)
incident_related_synonym_replaced["preprocessed_text_tokenized_no_hashtag"] = incident_related_back_translated.apply(clean_text, args=(["preprocessed_text_partly", "lose", "list"]), axis=1)

... OK (Got response in 0.39 seconds)
Writing emoji data to /Users/jorenwouters/.demoji/codes.json ...
... OK


## Create two datasets
Now, we will create the two datasets.

In [7]:
# From all the incident related tweets, only get the preprocessed_text_partly, hashtags and has_media
incident_related_data_rel = incident_related_data[["preprocessed_text", "preprocessed_text_no_hashtag", "preprocessed_text_tokenized", "preprocessed_text_tokenized_no_hashtag", "hashtags", "has_media", "has_Incident_Related"]]

# Merge incident-related tweets and backtranslated incident-related tweets into one dataframe
all_incident_related_data = pd.concat([incident_related_back_translated, incident_related_data_rel])

# From all the not incident related tweets, only get the preprocessed_text_partly, hashtags and has_media
not_incident_related_data_rel = not_incident_related_data[["preprocessed_text", "preprocessed_text_no_hashtag", "preprocessed_text_tokenized", "preprocessed_text_tokenized_no_hashtag", "hashtags", "has_media", "has_Incident_Related"]]

# Create the first balanced dataset
balanced_data = pd.concat([all_incident_related_data, not_incident_related_data_rel])

In [8]:
# Merge incident-related tweets and backtranslated incident-related tweets into one dataframe
all_incident_related_data_2 = pd.concat([incident_related_back_translated, incident_related_data_rel, incident_related_synonym_replaced])

# Create the second balanced dataset
balanced_data_2 = pd.concat([all_incident_related_data_2, not_incident_related_data_rel])

In [9]:
# Export the data
balanced_data.reset_index(inplace=True, drop=True)
balanced_data.to_csv("~/Documents/Github Repository/early-warning-twitter/Processed datasets/Tweets/01-06-2020-unique-tweets-amsterdam-demonstration-balanced.csv")
balanced_data.to_pickle("~/Documents/Github Repository/early-warning-twitter/Processed datasets/Tweets/01-06-2020-unique-tweets-amsterdam-demonstration-balanced.pkl")

balanced_data_2.reset_index(inplace=True, drop=True)
balanced_data_2.to_csv("~/Documents/Github Repository/early-warning-twitter/Processed datasets/Tweets/01-06-2020-unique-tweets-amsterdam-demonstration-balanced-2.csv")
balanced_data_2.to_pickle("~/Documents/Github Repository/early-warning-twitter/Processed datasets/Tweets/01-06-2020-unique-tweets-amsterdam-demonstration-balanced-2.pkl")

In [13]:
len(balanced_data_2)

6978

In [10]:
balanced_data["has_Incident_Related"].value_counts()

0    5070
1    1429
Name: has_Incident_Related, dtype: int64

In [11]:
balanced_data_2["has_Incident_Related"].value_counts()

0    5070
1    1908
Name: has_Incident_Related, dtype: int64